In [1]:
import pdfplumber
import re
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='your_api_key_here', 
    model_name="llama-3.1-70b-versatile"
)

In [3]:
# WebBaseLoader will accept the url and extract the data from that, ie web scraping

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.myntra.com/job-detail/?id=7431200002")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.



































Data Scientist

























About
Alum
Inclusion
Careers
Culture
Blog
Tech










Data Scientist
Bengaluru








Share









Apply



About Team
Myntra Data Science team delivers a large number of data science solutions for the company which are deployed at various customer touch points every quarter. The models create significant revenue and customer experience impact. The models involve real-time, near-real-time and offline solutions with varying latency requirements. The models are built using massive datasets. You will have the opportunity to be part of a rapidly growing organization and gain exposure to all the parts of a comprehensive ecommerce platform. You’ll also get to learn the intricacies of building models that serve millions of requests per second at sub second latency. 
The team takes pride in deploying solutions that not only leverage state of the art machine learning models like graph neural networks, diffusion model

In [4]:
from langchain_core.prompts import PromptTemplate
# (NO PREAMBLE) means dont give that initial text like Here is your response.
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

In [5]:
chain_extract = prompt_extract | llm    # this will form a langchain chain ie you are getting a prompt and passing it to LLM 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

# we got the json format of the job description

[
  {
    "role": "Data Scientist",
    "experience": "1+ years of relevant industry experience with a Bachelor’s degree or Master’s/PhD in Computer Science, Mathematics, Statistics/related fields",
    "skills": [
      "Python or one other high-level programming language",
      "Theoretical understanding of statistical models such as regression, clustering and ML algorithms such as decision trees, neural networks, etc.",
      "Machine learning frameworks like TensorFlow, PyTorch, or scikit-learn",
      "SQL and/or NoSQL databases"
    ],
    "description": "Design, develop and deploy machine learning models, algorithms and systems to solve complex business problems for Myntra Recsys, Search, Vision, SCM, Pricing, Forecasting, Trend and Virality prediction, Gen AI and other areas. Theoretical understanding and practise of machine learning and expertise in one or more of the topics, such as, NLP, Computer Vision, recommender systems and Optimisation."
  }
]


In [6]:
# but the type of it is string, we want json object so we will use JSON Parser
type(res.content)

str

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res1 = json_parser.parse(res.content)
json_res1

[{'role': 'Data Scientist',
  'experience': '1+ years of relevant industry experience with a Bachelor’s degree or Master’s/PhD in Computer Science, Mathematics, Statistics/related fields',
  'skills': ['Python or one other high-level programming language',
   'Theoretical understanding of statistical models such as regression, clustering and ML algorithms such as decision trees, neural networks, etc.',
   'Machine learning frameworks like TensorFlow, PyTorch, or scikit-learn',
   'SQL and/or NoSQL databases'],
  'description': 'Design, develop and deploy machine learning models, algorithms and systems to solve complex business problems for Myntra Recsys, Search, Vision, SCM, Pricing, Forecasting, Trend and Virality prediction, Gen AI and other areas. Theoretical understanding and practise of machine learning and expertise in one or more of the topics, such as, NLP, Computer Vision, recommender systems and Optimisation.'}]

In [9]:
# Check if the result is a list and extract the first dictionary
if isinstance(json_res1, list):
    json_res1 = json_res1[0]

In [10]:
job_description = json_res1

In [11]:
job_description

{'role': 'Data Scientist',
 'experience': '1+ years of relevant industry experience with a Bachelor’s degree or Master’s/PhD in Computer Science, Mathematics, Statistics/related fields',
 'skills': ['Python or one other high-level programming language',
  'Theoretical understanding of statistical models such as regression, clustering and ML algorithms such as decision trees, neural networks, etc.',
  'Machine learning frameworks like TensorFlow, PyTorch, or scikit-learn',
  'SQL and/or NoSQL databases'],
 'description': 'Design, develop and deploy machine learning models, algorithms and systems to solve complex business problems for Myntra Recsys, Search, Vision, SCM, Pricing, Forecasting, Trend and Virality prediction, Gen AI and other areas. Theoretical understanding and practise of machine learning and expertise in one or more of the topics, such as, NLP, Computer Vision, recommender systems and Optimisation.'}

In [12]:
import pdfplumber
import re

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        pages = [page.extract_text() for page in pdf.pages]
    all_text = "\n".join(pages) if pages else ""
    # print(all_text)
    return all_text

In [13]:
pdf_path = "C:/Users/Admin/Downloads/Mandar_Bhalerao_IISc.pdf"
pdf_data = extract_text_from_pdf(pdf_path)
# output_path = process_resume(pdf_path)
# print(f"Cold email prompt saved at: {output_path}")

In [41]:
pdf_data

'Mandar Bhalerao\n(cid:131) +91-8788519675 # mandarmanoj@iisc.ac.in # mandar02bhalerao@gmail.com (cid:239) linkedin ˆ HackerRank\nEducation\nIndian Institute of Science Aug 2023 – Jul 2025\nMaster of Technology - Computer Science and Automation CGPA : 7.30/10.0 Bangalore, Karnataka\nP.E.S. Modern College of Engineering Aug 2019 – Jul 2023\nBachelor of Engineering - Computer Engineering CGPA : 9.34/10.0 Pune, Maharashtra\nExperience\nNeuroPixel.AI — Deep Learning Research Intern May 2024 – July 2024\n• Worked on optimization of Stable Diffusion models to improve performance, achieving significant efficiency gains.\n• Implemented the Hyper-SD framework to enhance image synthesis efficiency by Knowledge Distillation techniques.\n• Combined the advantages of Trajectory Preserving and Reformulation Distillation techniques for faster inference.\n• Trained a Control Net for SDXL, resulting in a 30% improvement of the inference steps from the base SDXL model.\nWestern Union — Software Intern J

In [77]:
from langchain_core.prompts import PromptTemplate
# (NO PREAMBLE) means dont give that initial text like Here is your response.
prompt_extract = PromptTemplate.from_template(
        """
        ### PDF DATA OBTAINED FROM RESUME:
        {pdf_data}
        ### INSTRUCTION:
        The data is from the resume of a person.
        Your job is to extract all the details of this person and summarize it in 200 words, which includes name, education, experience, projects, skills, and achievements.
        ### (NO PREAMBLE):    
        """
)

In [78]:
chain_extract = prompt_extract | llm    # this will form a langchain chain ie you are getting a prompt and passing it to LLM 
res2 = chain_extract.invoke(input={'pdf_data':pdf_data})
# print(res.content)
summary = res2.content

In [79]:
print(summary)


Mandar Bhalerao is a highly skilled individual with a strong background in computer science and automation. He is currently pursuing a Master of Technology in Computer Science and Automation at the Indian Institute of Science, with a CGPA of 7.30/10.0. He holds a Bachelor of Engineering in Computer Engineering from P.E.S. Modern College of Engineering, with a CGPA of 9.34/10.0.

Mandar has gained valuable experience through internships at NeuroPixel.AI, Western Union, and Amazon Web Services. He worked on optimizing Stable Diffusion models, improving user experience design, and analyzing global transaction data. He also completed the Solutions Architect Project and gained insights into cloud architecture.

Mandar has worked on several projects, including a real estate price prediction model, optimizing performance of dilated convolution, and a movie recommendation system. He has demonstrated expertise in machine learning, deep learning, and natural language processing.

Mandar's techni

In [16]:
from langchain_core.prompts import PromptTemplate
# (NO PREAMBLE) means dont give that initial text like Here is your response.
prompt_extract = PromptTemplate.from_template(
        """
        ### PDF DATA OBTAINED FROM RESUME:
        {pdf_data}
        ### INSTRUCTION:
        The data is from the resume of a person.
        Your job is to extract all the details of this person and return them in JSON format containing the 
        following keys: `name`, `education`, `experience`, `projects`,`skills`, and `achievements`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

In [17]:
chain_extract = prompt_extract | llm    # this will form a langchain chain ie you are getting a prompt and passing it to LLM 
res = chain_extract.invoke(input={'pdf_data':pdf_data})
# print(res.content)

# we got the json format of the job description

In [18]:
type(res.content)

str

In [20]:
# res.content

In [21]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
# json_res

In [22]:
type(json_res)


dict

In [23]:
json_res

{'name': 'Mandar Bhalerao',
 'education': [{'institution': 'Indian Institute of Science',
   'degree': 'Master of Technology - Computer Science and Automation',
   'cgpa': '7.30/10.0',
   'duration': 'Aug 2023 – Jul 2025',
   'location': 'Bangalore, Karnataka'},
  {'institution': 'P.E.S. Modern College of Engineering',
   'degree': 'Bachelor of Engineering - Computer Engineering',
   'cgpa': '9.34/10.0',
   'duration': 'Aug 2019 – Jul 2023',
   'location': 'Pune, Maharashtra'}],
 'experience': [{'company': 'NeuroPixel.AI',
   'position': 'Deep Learning Research Intern',
   'duration': 'May 2024 – July 2024',
   'achievements': ['Worked on optimization of Stable Diffusion models to improve performance, achieving significant efficiency gains.',
    'Implemented the Hyper-SD framework to enhance image synthesis efficiency by Knowledge Distillation techniques.',
    'Combined the advantages of Trajectory Preserving and Reformulation Distillation techniques for faster inference.',
    'Trai

In [24]:
candidate_resume_details = json_res

In [25]:
type(candidate_resume_details)

dict

In [26]:
candidate_resume_details

{'name': 'Mandar Bhalerao',
 'education': [{'institution': 'Indian Institute of Science',
   'degree': 'Master of Technology - Computer Science and Automation',
   'cgpa': '7.30/10.0',
   'duration': 'Aug 2023 – Jul 2025',
   'location': 'Bangalore, Karnataka'},
  {'institution': 'P.E.S. Modern College of Engineering',
   'degree': 'Bachelor of Engineering - Computer Engineering',
   'cgpa': '9.34/10.0',
   'duration': 'Aug 2019 – Jul 2023',
   'location': 'Pune, Maharashtra'}],
 'experience': [{'company': 'NeuroPixel.AI',
   'position': 'Deep Learning Research Intern',
   'duration': 'May 2024 – July 2024',
   'achievements': ['Worked on optimization of Stable Diffusion models to improve performance, achieving significant efficiency gains.',
    'Implemented the Hyper-SD framework to enhance image synthesis efficiency by Knowledge Distillation techniques.',
    'Combined the advantages of Trajectory Preserving and Reformulation Distillation techniques for faster inference.',
    'Trai

In [27]:
job_description

{'role': 'Data Scientist',
 'experience': '1+ years of relevant industry experience with a Bachelor’s degree or Master’s/PhD in Computer Science, Mathematics, Statistics/related fields',
 'skills': ['Python or one other high-level programming language',
  'Theoretical understanding of statistical models such as regression, clustering and ML algorithms such as decision trees, neural networks, etc.',
  'Machine learning frameworks like TensorFlow, PyTorch, or scikit-learn',
  'SQL and/or NoSQL databases'],
 'description': 'Design, develop and deploy machine learning models, algorithms and systems to solve complex business problems for Myntra Recsys, Search, Vision, SCM, Pricing, Forecasting, Trend and Virality prediction, Gen AI and other areas. Theoretical understanding and practise of machine learning and expertise in one or more of the topics, such as, NLP, Computer Vision, recommender systems and Optimisation.'}

In [28]:
def generate_cold_email(details):
    # Extract name
    name = details.get('name', 'Candidate')

    # Extract education details
    education_list = details.get('education', [])
    if education_list:
        education_details = ', '.join([f"{edu.get('degree', 'Unknown degree')} from {edu.get('institution', 'Unknown institution')} ({edu.get('duration', 'Unknown duration')})" for edu in education_list])
    else:
        education_details = 'No education details provided'

    # Extract skills details
    skills_list = details.get('skills', [])
    if skills_list:
        skills_details = ', '.join([', '.join(skill.get('tools', [])) for skill in skills_list])
    else:
        skills_details = 'No skills listed'

    # Extract experience details
    experience_list = details.get('experience', [])
    if experience_list:
        experience_details = []
        for exp in experience_list:
            position = exp.get('position', 'Unknown position')
            company = exp.get('company', 'Unknown company')
            duration = exp.get('duration', 'Unknown duration')
            achievements = exp.get('achievements', [])
            achievements_details = ', '.join(achievements) if achievements else 'No achievements mentioned'
            experience_details.append(f"{position} at {company} ({duration}): {achievements_details}")
        experience_details = ', '.join(experience_details)
    else:
        experience_details = 'No experience provided'

    # Extract project details
    project_list = details.get('projects', [])
    if project_list:
        project_details = []
        for project in project_list:
            project_name = project.get('name', 'Unknown project')
            project_description = ', '.join(project.get('description', [])) if project.get('description') else 'No details provided'
            project_details.append(f"{project_name}: {project_description}")
        project_details = ', '.join(project_details)
    else:
        project_details = 'No projects listed'

    # Extract achievements details
    achievement_list = details.get('achievements', [])
    if achievement_list:
        achievement_details = ', '.join([f"{achieve.get('position', 'Unknown position')} - {achieve.get('description', 'Unknown achievement')}" for achieve in achievement_list])
    else:
        achievement_details = 'No achievements listed'

    email_prompt = f"""
    You are {name}, educated at {education_details}. Your skills include {skills_details}.

    You have experience as {experience_details}.
    
    Some of your key projects include {project_details}.
    
    Additionally, your achievements include {achievement_details}.
    
    Write a cold email to a potential employer or client, showcasing your skills, education, experience (including responsibilities and achievements), projects (with descriptions), and achievements. Explain how your background makes you an ideal candidate for their needs.

    Remember, you are {name}, ready to make a significant impact in your new role.
    """
    return email_prompt


In [29]:
generate_cold_email(candidate_resume_details)

AttributeError: 'str' object has no attribute 'get'

In [30]:
def generate_cold_email(details):
    # Extract name
    name = details.get('name', 'Candidate')

    # Extract education details
    education_list = details.get('education', [])
    if education_list:
        education_details = ', '.join([f"{edu.get('degree', 'Unknown degree')} from {edu.get('institution', 'Unknown institution')} ({edu.get('duration', 'Unknown duration')})" for edu in education_list])
    else:
        education_details = 'No education details provided'

    # Extract skills details
    skills_list = details.get('skills', [])
    if skills_list:
        skills_details = ', '.join([
            ', '.join(skill.get('tools', [])) if isinstance(skill, dict) else str(skill) 
            for skill in skills_list
        ])
    else:
        skills_details = 'No skills listed'

    # Extract experience details
    experience_list = details.get('experience', [])
    if experience_list:
        experience_details = []
        for exp in experience_list:
            position = exp.get('position', 'Unknown position')
            company = exp.get('company', 'Unknown company')
            duration = exp.get('duration', 'Unknown duration')
            achievements = exp.get('achievements', [])
            achievements_details = ', '.join(achievements) if achievements else 'No achievements mentioned'
            experience_details.append(f"{position} at {company} ({duration}): {achievements_details}")
        experience_details = ', '.join(experience_details)
    else:
        experience_details = 'No experience provided'

    # Extract project details
    project_list = details.get('projects', [])
    if project_list:
        project_details = []
        for project in project_list:
            project_name = project.get('name', 'Unknown project')
            project_description = ', '.join(project.get('description', [])) if project.get('description') else 'No details provided'
            project_details.append(f"{project_name}: {project_description}")
        project_details = ', '.join(project_details)
    else:
        project_details = 'No projects listed'

    # Extract achievements details
    achievement_list = details.get('achievements', [])
    if achievement_list:
        achievement_details = ', '.join([f"{achieve.get('position', 'Unknown position')} - {achieve.get('description', 'Unknown achievement')}" for achieve in achievement_list])
    else:
        achievement_details = 'No achievements listed'

    email_prompt = f"""
    You are {name}, educated at {education_details}. Your skills include {skills_details}.
    
    You have experience as {experience_details}.
    
    Some of your key projects include {project_details}.
    
    Additionally, your achievements include {achievement_details}.
    
    Write a cold email to a potential employer or client, showcasing your skills, education, experience (including responsibilities and achievements), projects (with descriptions), and achievements. Explain how your background makes you an ideal candidate for their needs.

    Remember, you are {name}, ready to make a significant impact in your new role.
    """
    return email_prompt


In [31]:
generate_cold_email(candidate_resume_details)

'\n    You are Mandar Bhalerao, educated at Master of Technology - Computer Science and Automation from Indian Institute of Science (Aug 2023 – Jul 2025), Bachelor of Engineering - Computer Engineering from P.E.S. Modern College of Engineering (Aug 2019 – Jul 2023). Your skills include languages, developerTools, technical.\n    \n    You have experience as Deep Learning Research Intern at NeuroPixel.AI (May 2024 – July 2024): Worked on optimization of Stable Diffusion models to improve performance, achieving significant efficiency gains., Implemented the Hyper-SD framework to enhance image synthesis efficiency by Knowledge Distillation techniques., Combined the advantages of Trajectory Preserving and Reformulation Distillation techniques for faster inference., Trained a Control Net for SDXL, resulting in a 30% improvement of the inference steps from the base SDXL model., Software Intern at Western Union (Jan 2023 – Jun 2023): Engaged with Quantum Metric to enrich powerful UX analysis, 

In [32]:
def generate_cold_email(details):
    # Extract name
    name = details.get('name', 'Candidate')

    # Extract education details
    education_list = details.get('education', [])
    if education_list:
        education_details = ', '.join([f"{edu.get('degree', 'Degree not specified')} from {edu.get('institution', 'Institution not specified')} ({edu.get('duration', 'Duration not specified')})" for edu in education_list])
    else:
        education_details = 'No education details provided'

    # Extract skills details
    skills = details.get('skills', {})
    skills_details = []
    if isinstance(skills, dict):
        for category, tools in skills.items():
            if isinstance(tools, list):
                skills_details.append(f"{category.capitalize()}: {', '.join(tools)}")
            else:
                skills_details.append(f"{category.capitalize()}: {tools}")
    elif isinstance(skills, list):
        skills_details.append(', '.join(skills))
    else:
        skills_details = 'No skills listed'
    
    skills_details = '; '.join(skills_details) if skills_details else 'No skills listed'

    # Extract experience details
    experience_list = details.get('experience', [])
    if experience_list:
        experience_details = []
        for exp in experience_list:
            position = exp.get('position', 'Position not specified')
            company = exp.get('company', 'Company not specified')
            duration = exp.get('duration', 'Duration not specified')
            achievements = exp.get('achievements', [])
            if isinstance(achievements, list):
                achievements_details = ', '.join(achievements) if achievements else 'No achievements mentioned'
            else:
                achievements_details = achievements
            experience_details.append(f"{position} at {company} ({duration}): {achievements_details}")
        experience_details = '; '.join(experience_details)
    else:
        experience_details = 'No experience provided'

    # Extract project details
    project_list = details.get('projects', [])
    if project_list:
        project_details = []
        for project in project_list:
            project_name = project.get('name', 'Project name not specified')
            technologies = project.get('technologies', 'Technologies not specified')
            achievements = project.get('achievements', [])
            if isinstance(achievements, list):
                project_achievements = ', '.join(achievements) if achievements else 'No details provided'
            else:
                project_achievements = achievements
            project_details.append(f"{project_name} (Technologies: {technologies}): {project_achievements}")
        project_details = '; '.join(project_details)
    else:
        project_details = 'No projects listed'

    # Extract achievements details
    achievement_list = details.get('achievements', [])
    if achievement_list:
        achievement_details = []
        for achieve in achievement_list:
            position = achieve.get('position', 'Position not specified')
            achievement_desc = ', '.join([f"{key.capitalize()}: {value}" for key, value in achieve.items() if key != 'position'])
            achievement_details.append(f"{position} - {achievement_desc}")
        achievement_details = '; '.join(achievement_details)
    else:
        achievement_details = 'No achievements listed'

    email_prompt = f"""
    You are {name}, educated at {education_details}. Your skills include {skills_details}.
    
    You have experience as {experience_details}.
    
    Some of your key projects include {project_details}.
    
    Additionally, your achievements include {achievement_details}.
    
    Write a cold email to a potential employer or client, showcasing your skills, education, experience (including responsibilities and achievements), projects (with descriptions), and achievements. Explain how your background makes you an ideal candidate for their needs.

    Remember, you are {name}, ready to make a significant impact in your new role.
    """
    return email_prompt

In [33]:
generate_cold_email(candidate_resume_details)


"\n    You are Mandar Bhalerao, educated at Master of Technology - Computer Science and Automation from Indian Institute of Science (Aug 2023 – Jul 2025), Bachelor of Engineering - Computer Engineering from P.E.S. Modern College of Engineering (Aug 2019 – Jul 2023). Your skills include Languages: Python, C++; Developertools: VS Code, Jupyter Notebook, Google Colab; Technical: Neural Networks, Machine Learning, Deep Learning, Gen AI, Natural Language Processing (NLP).\n    \n    You have experience as Deep Learning Research Intern at NeuroPixel.AI (May 2024 – July 2024): Worked on optimization of Stable Diffusion models to improve performance, achieving significant efficiency gains., Implemented the Hyper-SD framework to enhance image synthesis efficiency by Knowledge Distillation techniques., Combined the advantages of Trajectory Preserving and Reformulation Distillation techniques for faster inference., Trained a Control Net for SDXL, resulting in a 30% improvement of the inference st

In [35]:
resume_details = generate_cold_email(candidate_resume_details)
type(resume_details)

str

In [36]:
# this is prompt template for writing an email

prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        Introduce yourself from the below details
        {resume_details}
        End the email with Name and Designation. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

In [54]:
# job_description

In [53]:
# candidate_resume_details

In [40]:
# again creating a chain of prompt_email and llm
# invoking the chain by passing the parameter of job_description and link_list

# chain_email = prompt_email | llm
# res = chain_email.invoke({"job_description": str(job_description), "resume_details": candidate_resume_details})
# print(res.content)

I am a highly motivated and detail-oriented Data Scientist with a strong educational background in Computer Science and Automation. I hold a Master of Technology degree from the Indian Institute of Science and a Bachelor of Engineering degree from P.E.S. Modern College of Engineering.

With over a year of industry experience, I have worked as a Deep Learning Research Intern at NeuroPixel.AI, where I optimized Stable Diffusion models to improve performance, achieving significant efficiency gains. I also worked as a Software Intern at Western Union, where I analyzed global transaction data, identified bugs, and implemented solutions that boosted conversion rates by 10%. Additionally, I have completed internships at Amazon Web Services, where I acquired foundational skills in AWS and utilized essential tools and services to support scalable cloud architectures.

I have a strong technical skillset, with expertise in Python, C++, Neural Networks, Machine Learning, Deep Learning, Gen AI, and

In [96]:
# this is prompt template for writing an email

prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        This is a job description
        
        {job_description}

        ### INSTRUCTION:
        These are the person's details.
        {summary}
        Consider yourself as this person. 
        
        Introduce yourself in an engaging way from above with your name from the above details and your current designation.  
        Try to find some similar things in the job description with your details. Mention those things which are similar. Mention your experience details.
        Your job is to write a cold email to the hiring manager regarding the job mentioned above describing the capability of you 
        in fulfilling their needs.
        End the email with Name and Current place where your are working or studying. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

In [97]:
# again creating a chain of prompt_email and llm
# invoking the chain by passing the parameter of job_description and link_list

chain_email = prompt_email | llm
res5 = chain_email.invoke({"job_description": str(job_description), "summary": summary})
print(res5.content)

Subject: Application for Data Scientist Role at Myntra

Dear Hiring Manager,

I'm Mandar Bhalerao, a highly skilled and enthusiastic individual with a strong background in computer science and automation, currently pursuing my Master of Technology in Computer Science and Automation at the Indian Institute of Science.

As I came across the Data Scientist role at Myntra, I was excited to see the alignment between the job requirements and my skills. With a solid foundation in computer science and a CGPA of 7.30/10.0 in my current program, I'm confident in my ability to design, develop, and deploy machine learning models to solve complex business problems.

I noticed that the job description mentions expertise in machine learning frameworks like TensorFlow, PyTorch, or scikit-learn, which I've gained experience with through my projects, including a real estate price prediction model and optimizing performance of dilated convolution. Additionally, my proficiency in Python, a high-level prog